In [1]:
!nvidia-smi

Tue Apr  9 16:02:43 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.52                 Driver Version: 551.52         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| 44%   27C    P8             13W /  125W |     676MiB /   6144MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import json
import re
from pprint import pprint
import numpy as np
import pandas as pd
#import bitsandbytes

# %load_ext cudf.pandas  # pandas operations now use the GPU!
import torch
from peft import LoraConfig, PeftModel
from huggingface_hub import notebook_login
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from trl import SFTTrainer
from trl import setup_chat_format
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

DEVICE =  "cuda: 0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf"
DEVICE

'cuda: 0'

In [ ]:
from datasets import load_dataset, Dataset, Value, ClassLabel, Features

my_dataset = load_dataset("./Emotion_Dataset", sep=",")
# Creating a ClassLabel Object
df = my_dataset["train"].to_pandas()
labels = ['sadness','joy','love','anger', 'fear', 'surprise']
ClassLabels = ClassLabel(num_classes=len(labels), names=labels)

# Mapping Labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['label'])
    return example

my_dataset= my_dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
my_dataset = my_dataset.cast_column('label', ClassLabels)

my_dataset

In [ ]:
dataset_df = my_dataset["train"].to_pandas()
dataset_df

In [ ]:
features = my_dataset["train"].features
features

In [ ]:
id2label = {idx:features["label"].int2str(idx) for idx in range(6)}
id2label

In [ ]:
label2id = {v:k for k,v in id2label.items()}
label2id

In [ ]:
dataset_df["label"].value_counts(normalize=True).sort_index()

In [ ]:
class_weights = (1 - (dataset_df["label"].value_counts().sort_index() / len(dataset_df))).values #assigning high weight to the rare classes and low weight to the common classes to balance it out as there is an imbalance in the dataset
class_weights


class_weights = torch.from_numpy(class_weights).float()
class_weights = class_weights.to("cuda")
class_weights